[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://drive.google.com/drive/folders/1IrwoNrb3AWLAhAqjlAkJNYa39p9eT9ui?usp=sharing)


# Flotorch Agent Latency Evaluation (Flotorch-Assistant Use Case)

This notebook demonstrates how to measure and analyze the **agent latency** of a **Flotorch ADK agent** (configured as a **Flotorch-Assistant** that provides context-aware responses using Flotorch documentation via RAG) using the **Flotorch Eval** framework.

The evaluation relies on **OpenTelemetry Traces** generated during the agent's run to provide a detailed breakdown of execution time across all steps (LLM calls, tool execution, etc.).

---

## Key Concepts

* **Flotorch-Assistant**: An agent designed to provide context-aware responses using Flotorch documentation via RAG (Retrieval-Augmented Generation).
* **OpenTelemetry Traces**: Detailed records of the agent's execution steps (spans) used to measure time taken by different operations.
* **LatencyMetric**: A Flotorch Eval metric that extracts and summarizes latency information from the execution traces (agent trajectories). The evaluation metric used is **latency_summary**.

### Architecture Overview

![Workflow Diagram](diagrams/01_LatencyMetric_Workflow_Diagram.drawio.png)
*Figure : Detailed workflow diagram showing the step-by-step process of latency evaluation from agent execution through trace collection to metric computation.*

---

## Requirements

* Flotorch account with configured models and a knowledge base containing Flotorch documentation.
* Valid Flotorch API key and gateway base URL.
* Agent configured with OpenTelemetry tracing enabled.

---

## Agent Setup in Flotorch Console

**Important**: Before running this notebook, you need to create an agent in the Flotorch Console. This section provides step-by-step instructions on how to set up the agent.

### Step 1: Access Flotorch Console

1. **Log in to Flotorch Console**:
   - Navigate to your Flotorch Console (e.g., `https://dev-console.flotorch.cloud`)
   - Ensure you have the necessary permissions to create agents

2. **Navigate to Agents Section**:
   - Click on **"Agents"** in the left sidebar
   - You should see the "Agent Builder" option selected

### Step 2: Create New Agent

1. **Click "Create FloTorch Agent"**:
   - Look for the blue **"+ Create FloTorch Agent"** button in the top right corner
   - Click it to start creating a new agent

2. **Agent Configuration**:
   - **Agent Name**: Choose a unique name for your agent (e.g., `flotorch-agent`)
     - **Important**: The name should only contain alphanumeric characters and dashes (a-z, A-Z, 0-9, -)
     - **Note**: Copy this agent name - you'll need to use it in the `agent_name` variable later
   - **Description** (Optional): Add a description if desired

### Step 3: Configure Agent Details

After creating the agent, you'll be directed to the agent configuration page. Configure the following:

#### Required Configuration:

1. **Model** (`* Model`):
   - **Required**: Select a model from the available options
   - Example: `gpt-model` or any available model from your Flotorch gateway
   - Click the edit icon to configure

2. **Agent Details** (`* Agent Details`):
   - **Required**: Configure agent details
   - **System Prompt**: Copy and paste the following system prompt:

You are the Flotorch Support Agent. Your purpose is to assist users with questions about Flotorch by providing accurate, context-aware answers grounded strictly in official Flotorch documentation.

Use the kb_tool to look up any information you are not fully certain about. Always query the knowledge base before answering if clarification, validation, or exact details are needed.

Guidelines:
Base every response on retrieved knowledge from kb_tool.
If multiple document fragments are returned, synthesize them into a single clear answer.
Never invent features, APIs, capabilities, or behaviors not found in the retrieved documentation.
If the knowledge base lacks information, say so clearly and provide a safe, general-purpose suggestion.
Provide concise, helpful explanations with step-by-step details when relevant.
Maintain a professional, technical-support tone.

Your outputs should always reflect the most accurate Flotorch information available via the kb_tool


   - **Goal**: Copy and paste the following goal:
   
To deliver precise, context-aware technical support for Flotorch by retrieving and grounding all answers in the Flotorch knowledge base using kb_tool, ensuring correctness, clarity, and factual alignment with documentation.


#### Optional Configuration:

1. **Tools**:
   - Tools will be added programmatically via the notebook (see Section 8)
   - You can leave this as "Not Configured" in the console

2. **Input Schema**:
   - Optional: Leave as "Not Configured" for this use case

3. **Output Schema**:
   - Optional: Leave as "Not Configured" for this use case

### Step 4: Publish the Agent

1. **Review Configuration**:
   - Ensure the Model and Agent Details are configured correctly
   - Verify the System Prompt and Goal are set

2. **Publish Agent**:
   - After configuration, click **"Publish"** or **"Make a revision"** to publish the agent
   - Once published, the agent will have a version number (e.g., v1)

3. **Note the Agent Name**:
   - **Important**: Copy the exact agent name you used when creating the agent
   - You will need to replace `<your_agent_name>` in the `agent_name` variable in Section 2.1 (Global Provider Models and Agent Configuration)

### Step 5: Update Notebook Configuration

1. **Update Agent Name**:
   - Navigate to Section 2.1 in this notebook
   - Find the `agent_name` variable
   - Replace `<your_agent_name>` with the exact agent name you created in the console

**Example**:
- If you created an agent named `flotorch-agent` in the console
- Set `agent_name = "flotorch-agent"` in the notebook

### Summary of Required vs Optional Settings

| Setting | Required/Optional | Value |
|---------|------------------|-------|
| **Agent Name** | **Required** | Choose a unique name (copy it for notebook) |
| **Model** | **Required** | Select from available models |
| **System Prompt** | **Required** | Use the system prompt provided above |
| **Goal** | **Required** | Use the goal provided above |
| **Tools** | **Optional** | Will be added via notebook code |
| **Input Schema** | **Optional** | Can leave as "Not Configured" |
| **Output Schema** | **Optional** | Can leave as "Not Configured" |

**Note**: The tools (Knowledge Base, Web Search, Weather, News) will be added to the agent programmatically in the notebook code, so you don't need to configure them manually in the console.

---


## 1. Setup and Installation

### Purpose
Install the necessary packages for the Flotorch Evaluation framework required for agent latency evaluation.

### Key Components
- **`flotorch-eval`**: Flotorch evaluation framework with all dependencies for latency metrics



In [ ]:
# Install Flotorch Eval packages
# flotorch-eval: Flotorch evaluation framework with all dependencies

%pip install flotorch-eval==2.0.0b1 flotorch[adk]==3.1.0b1

## 2.Authentication and Credentials

### Purpose
Configure your Flotorch API credentials and gateway URL for authentication.

### Key Components
This cell configures the essential authentication and connection parameters:

**Authentication Parameters**:

| Parameter | Description | Example |
|-----------|-------------|---------|
| `FLOTORCH_API_KEY` | Your API authentication key (found in your Flotorch Console). Securely entered using `getpass` to avoid displaying in the notebook | `sk_...` |
| `FLOTORCH_BASE_URL` | Your Flotorch gateway endpoint URL | `https://dev-console.flotorch.cloud` |

**Note**: Use secure credential management in production environments.


In [ ]:
import getpass  # Securely prompt without echoing in notebooks

# authentication for Flotorch access
try:
    FLOTORCH_API_KEY = getpass.getpass("Paste your API key here: ")  
    print(f"✓ FLOTORCH_API_KEY set successfully")
except getpass.GetPassWarning as e:
    print(f"Warning: {e}")
    FLOTORCH_API_KEY = ""
    print(f"✗ FLOTORCH_API_KEY not set")

FLOTORCH_BASE_URL = input("Paste your Flotorch Base URL here: ")   # https://dev-gateway.flotorch.cloud
print(f"✓ FLOTORCH_BASE_URL set: {FLOTORCH_BASE_URL}")

print("✓ All credentials configured successfully!")

### 2.1. Global Provider Models and Agent Configuration

### Purpose
Define available models from the Flotorch gateway and configure agent-specific parameters.

### Key Components

**Global Provider Models**: These are the available models from the Flotorch gateway that can be used for evaluation and agent operations:

| Model Variable | Model Name | Description |
|----------------|------------|-------------|
| `MODEL_CLAUDE_HAIKU` | `flotorch/flotorch-claude-haiku-4-5` | Claude Haiku model via Flotorch gateway |
| `MODEL_CLAUDE_SONNET` | `flotorch/flotorch-claude-sonnet-3-5-v2` | Claude Sonnet model via Flotorch gateway |
| `MODEL_AWS_NOVA_PRO` | `flotorch/flotorch-aws-nova-pro` | AWS Nova Pro model via Flotorch gateway |
| `MODEL_AWS_NOVA_LITE` | `flotorch/flotorch-aws-nova-lite` | AWS Nova Lite model via Flotorch gateway |
| `MODEL_AWS_NOVA_MICRO` | `flotorch/flotorch-aws-nova-micro` | AWS Nova Micro model via Flotorch gateway |

**Agent Configuration Parameters**:

| Parameter | Description | Example |
|-----------|-------------|---------|
| `KNOWLEDGE_BASE` | The ID of your Flotorch Knowledge Base | `knowledge-base` |
| `agent_name` | The name of your Flotorch ADK agent | `flotorch-agent` |
| `default_evaluator` | The LLM model used for evaluation (can use MODEL_* variables above) | `MODEL_CLAUDE_SONNET` or `flotorch/flotorch-model` |
| `app_name` | The application name identifier | `agent-evaluation-app-name_01` |
| `user_id` | The user identifier | `agent-evaliation-user-01` |


In [ ]:
# ============================================================================
# Global Provider Models (Flotorch Gateway Models)
# ============================================================================
# These models are available from the Flotorch gateway and can be used
# for evaluation, agent operations, and other tasks.

MODEL_CLAUDE_HAIKU = "flotorch/flotorch-claude-haiku-4-5"
MODEL_CLAUDE_SONNET = "flotorch/flotorch-claude-sonnet-3-5-v2"
MODEL_AWS_NOVA_PRO = "flotorch/flotorch-aws-nova-pro"
MODEL_AWS_NOVA_LITE = "flotorch/flotorch-aws-nova-lite"
MODEL_AWS_NOVA_MICRO = "flotorch/flotorch-aws-nova-micro"

print("✓ Global provider models defined")

KNOWLEDGE_BASE = "<your_knowledge_base_id>"  # The ID of your Flotorch Knowledge Base                          || ex : knowledge-base
agent_name = "<your_agent_name>"  # The name of your Flotorch ADK agent                                        || ex : flotorch-agent
default_evaluator = MODEL_CLAUDE_HAIKU  # The LLM model used for evaluation. Can be modified to use any MODEL_* constant above (e.g., MODEL_CLAUDE_SONNET, MODEL_AWS_NOVA_PRO)
app_name = "<your_app_name>"  # The application name identifier                                                || ex : agent-evaluation-app-name_01
user_id = "<your_user_id>"  # The user identifier                                                              || ex : agent-evaliation-user-01

print("✓ Agent Configuration Parameter defined ")


## 3. Import Required Libraries

### Purpose
Import all required components for evaluating the Flotorch Support Agent latency using Flotorch Eval.

### Key Components
- **`AgentEvaluator`**: Core client for agent evaluation orchestration and trace fetching
- **`LatencyMetric`**: Flotorch Eval metric that extracts and summarizes latency information from execution traces
- **`FlotorchADKAgent`**: Creates and configures Flotorch ADK agents with custom tools and tracing
- **`FlotorchADKSession`**: Manages agent sessions for multi-turn conversations
- **`FlotorchVectorStore`**: Connects to the Flotorch knowledge base for RAG-based context retrieval
- **`extract_vectorstore_texts`**: Utility helper for extracting text from vector-store search results
- **`Runner`**: Executes agent queries and coordinates the agent execution flow
- **`FunctionTool`**: Wraps Python functions as tools that can be used by the agent
- **`types`**: Google ADK types for creating message content and handling agent events
- **`pandas`**: Data manipulation and display for formatted results tables
- **`display`**: IPython display utility for rendering formatted outputs

In [ ]:
# Required imports
# Flotorch Eval components
from flotorch_eval.agent_eval.core.client import AgentEvaluator
from flotorch_eval.agent_eval.metrics.latency_metrics import LatencyMetric

# Flotorch ADK components
from flotorch.adk.agent import FlotorchADKAgent
from flotorch.adk.sessions import FlotorchADKSession
from flotorch.sdk.memory import FlotorchVectorStore
from flotorch.sdk.utils.memory_utils import extract_vectorstore_texts

# Google ADK components
from google.adk.runners import Runner
from google.adk.tools import FunctionTool
from google.genai import types

# Utilities
import pandas as pd
from IPython.display import display

print("✓ Imported necessary libraries successfully")

## 4. Flotorch Support Agent Setup

### Purpose
Set up the Flotorch Support Agent with OpenTelemetry tracing enabled to capture detailed execution data for latency evaluation.

### Key Components
1. **FlotorchADKAgent** (`agent_client`):
   - Initializes the agent for providing context-aware responses using Flotorch documentation via RAG
   - Configures `tracer_config` with `enabled: True` and `sampling_rate: 1` to capture 100% of traces
   - Essential for evaluation as traces contain complete latency information
2. **FlotorchADKSession** (`session_service`): Manages agent sessions for multi-turn conversations
3. **Runner** (`runner`): Executes agent queries and coordinates the agent execution flow

These components work together to run the Flotorch Support Agent and generate OpenTelemetry traces for latency analysis.

### Custom Tool: RAG-based Knowledge Base Search

The Flotorch Support Agent uses a custom tool (`kb_tool`) that integrates with the Flotorch knowledge base to retrieve relevant documentation context. This tool:
- Accepts a query string as input
- Performs semantic search in the vectorstore using FlotorchVectorStore
- Extracts and returns relevant text chunks from Flotorch documentation
- Handles errors gracefully with structured error responses

The tool is wrapped as a `FunctionTool` that can be used by the agent to provide context-aware responses using Flotorch documentation via RAG.

In [ ]:
# Initialize the Flotorch Vector Store (Knowledge Base)
kb = FlotorchVectorStore(
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_BASE_URL,
    vectorstore_id=KNOWLEDGE_BASE
)

def kb_tool(query: str) -> dict:
    """
    Custom tool for searching the Flotorch knowledge base.
    Performs semantic search and returns relevant text chunks.
    """
    try:
        if not query:
            return {
                "success": False,
                "results": [],
                "error": "Empty query provided."
            }

        # Perform semantic search in vectorstore
        context = kb.search(query)
        results = extract_vectorstore_texts(context)

        return {
            "success": True,
            "results": results,
            "error": None
        }

    except Exception as e:
        return {
            "success": False,
            "results": [],
            "error": str(e)
        }


# Register the custom tool
tools = [FunctionTool(kb_tool)]

print("✓ Knowledge base tool setup completed!")


## 5. Agent and Runner Initialization

### Purpose
Set up the Flotorch ADK Agent and Runner with OpenTelemetry tracing enabled to capture detailed latency data for evaluation.

### Key Components
1. **FlotorchADKAgent** (`agent_client`):
   - Initializes the agent with custom RAG tools
   - Configures `tracer_config` with `enabled: True` and `sampling_rate: 1` to capture 100% of traces
   - Essential for latency measurement as traces contain timing information
2. **FlotorchADKSession** (`session_service`): Manages agent sessions for multi-turn conversations
3. **Runner** (`runner`): Executes agent queries and coordinates the agent execution flow

These components work together to run the Flotorch Support Agent and generate OpenTelemetry traces for latency analysis.

In [ ]:
# Initialize Flotorch ADK Agent with tracing enabled
agent_client = FlotorchADKAgent(
    agent_name=agent_name,
    custom_tools=tools,
    base_url=FLOTORCH_BASE_URL,
    api_key=FLOTORCH_API_KEY,
    tracer_config={
        "enabled": True,                                                   # Enable tracing for latency measurement
        "endpoint": "https://dev-observability.flotorch.cloud/v1/traces",  # Dev observability OTLP HTTP endpoint (used by QA)
        "sampling_rate": 1                                                 # Sample 100% of traces
    }
)
agent = agent_client.get_agent()

# Initialize session service
session_service = FlotorchADKSession(
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_BASE_URL,
)

# Create the ADK Runner to execute agent queries
runner = Runner(
    agent=agent,
    app_name=app_name,
    session_service=session_service
)

print("✓ Agent and runner and session setup completed!")

## 6. Helper Function for Running a Query

### Purpose
Define a helper function that executes a single-turn query with the agent and extracts the final response. The agent execution is automatically traced for latency measurement.

### Functionality
The `run_single_turn` function:
- Accepts a `Runner`, query string, session ID, and user ID as parameters
- Creates a user message using Google ADK types
- Executes the query through the runner
- Iterates through events to find and return the final agent response
- Returns a fallback message if no response is found

This function simplifies the process of running queries and ensures trace generation during execution.

In [ ]:
def run_single_turn(runner: Runner, query: str, session_id: str, user_id: str) -> str:
    """
    Execute a single-turn query with the agent and return the final response.
    The agent execution is traced automatically.
    """
    content = types.Content(role="user", parts=[types.Part(text=query)])
    events = runner.run(user_id=user_id, session_id=session_id, new_message=content)

    # Extract the final response
    for event in events:
        if event.is_final_response() and event.content and event.content.parts:
            return event.content.parts[0].text
    return "No response from agent."

print("✓ Helper function 'run_single_turn' defined successfully")

## 7. Define Query

### Purpose
Define the sample query that will be executed by the Flotorch Support Agent to generate OpenTelemetry traces for latency evaluation.

### Key Components
- **`query`**: A sample question that will be processed by the agent
  - This query will trigger the agent to use RAG (knowledge base search) and generate a response
  - The execution will be automatically traced to capture latency information
  - Example: "How to create agents in flotorch console?"

The query can be modified to test different scenarios and measure latency for various types of questions.


In [ ]:
# Execute the query to generate traces

query = "How to create agents in flotorch console?"

print(f"✓ Query defined: {query}")

## 8. Run the Query and Get Trace ID

### Purpose
Execute a sample query with the Flotorch Support Agent to generate OpenTelemetry traces that contain latency data for evaluation.

### Process
1. **Create Session**: Initialize a new session for the agent interaction
2. **Execute Query**: Run a sample query ("How to create agents in flotorch console?") through the agent
3. **Retrieve Trace IDs**: Extract the generated trace IDs from the agent client
4. **Display Results**: Print the agent response and trace ID for verification

The execution automatically generates OpenTelemetry traces that record timing information for all agent operations (LLM calls, tool executions, etc.), which will be used for latency evaluation.

In [ ]:
# Create a new session
session = await runner.session_service.create_session(
    app_name=app_name,
    user_id=user_id,
)
print(f"Session created: {session.id}")

response = run_single_turn(
    runner=runner,
    query=query,
    session_id=session.id,
    user_id=user_id
)

# Retrieve the generated trace IDs
trace_ids = agent_client.get_tracer_ids()
print("Agent Response:")
print(response[:200] + "..." if len(response) > 200 else response)
print(f"Found {len(trace_ids)} trace(s). First trace ID: {trace_ids[0] if trace_ids else 'N/A'}")

print(f"✓ Trace IDs retrieved: {len(trace_ids)} trace(s) generated")

## 9. Latency Evaluation with Flotorch Eval

### Purpose
Initialize the `AgentEvaluator`, fetch the OpenTelemetry trace, and run the `LatencyMetric` to compute the performance breakdown. The evaluation metric **latency_summary** provides detailed latency statistics for the Flotorch Support Agent.

### Key Components
1. **LatencyMetric**: Initializes the latency metric that will analyze trace data
2. **AgentEvaluator** (`client`):
   - Connects to the Flotorch gateway using API credentials
   - Configured with a default evaluator model
   - Provides methods to fetch and evaluate traces
3. **Trace Fetching**: Retrieves the complete trace data using the trace ID generated during agent execution

The fetched trace contains detailed timing information for all agent operations, which will be analyzed by the LatencyMetric to compute the latency_summary.

In [ ]:
# Initialize the LatencyMetric
metrics = [LatencyMetric()]

# Initialize the AgentEvaluator client
client = AgentEvaluator(
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_BASE_URL,
    default_evaluator=default_evaluator
)
print("✓ AgentEvaluator client initialized successfully")

traces = None
if trace_ids:
    # Fetch the trace data from the Flotorch gateway
    traces = client.fetch_traces(trace_ids[0])
else:
    print("✗ No trace IDs found to fetch.")

## 10. Run Evaluation

### Purpose
Execute the latency evaluation by processing the fetched OpenTelemetry trace using the LatencyMetric to compute performance breakdown.

### Process
- Calls `client.evaluate()` with the trace data and LatencyMetric
- The evaluator processes the trace to extract timing information for all agent steps
- Computes the **latency_summary** metric which includes:
  - Total latency for root steps
  - Average step latency
  - Detailed breakdown by operation type (LLM calls, tool execution, etc.)
- Returns evaluation results with trajectory ID and metric scores

This step generates the latency analysis that will be displayed in the next section.

In [ ]:
if 'traces' in locals() and traces:
    # Evaluate the trace using the LatencyMetric
    results = await client.evaluate(
        trace=traces,
        metrics=metrics
    )

    print("✓ Evaluation completed successfully!")
else:
    print("Cannot evaluate: No traces were available.")


## 11. Display and Interpret Results

### Purpose
Define helper functions to format and display the evaluation output clearly, showing the latency_summary metric results in a readable format.
### Functionality
The `display_metrics` function:
- Extracts the `latency_summary` metric from evaluation results
- Formats the latency breakdown with step names and timing in milliseconds
- Creates a structured display showing:
  - Total Latency (Root Steps)
  - Average Root Step Latency
  - Detailed latency breakdown for each operation
- Uses pandas DataFrame with styled formatting for clean presentation

This function provides a user-friendly way to visualize latency performance metrics.

In [ ]:
def display_metrics(result):
    """
    Display latency metrics in a formatted table, including total and step breakdown.
    """
    # Find the latency_summary metric
    metric = next((m for m in result.scores if m.name == "latency_summary"), None)
    if not metric:
        print("No latency_summary metric found.")
        return

    # Extract metric details
    d = metric.details

    # Build the latency breakdown string
    breakdown = "\n".join(
        f"    - {s['step_name']}: {s['latency_ms']} ms"
        for s in d.get("latency_breakdown", [])
    )

    # Format the details string
    details = (
        f"Total Latency (Root Steps): {d.get('total_latency_ms')} ms\n"
        f"Average Root Step Latency: {d.get('average_step_latency_ms')} ms\n"
        f"Latency Breakdown:\n{breakdown}"
    )

    # Create DataFrame for display
    df = pd.DataFrame([{
        "Metric": metric.name,
        "Score": f"{metric.score:.2f}",
        "Details": details
    }])

    # Display DataFrame with multiline support
    display(df.style.set_properties(
        subset=['Details'],
        **{'white-space': 'pre-wrap', 'text-align': 'left'}
    ))


print("✓ Display metrics function defined successfully")


## 12. View Latency Results

### Purpose
Display the latency evaluation results in a formatted table showing the complete performance breakdown for the Flotorch Support Agent.

### Output
The displayed table includes:
- **Metric**: The evaluation metric name (latency_summary)
- **Score**: The overall latency score
- **Details**: Comprehensive breakdown showing:
  - Total Latency (Root Steps) in milliseconds
  - Average Root Step Latency in milliseconds
  - Individual step latencies (e.g., call_llm, execute_tool)

This visualization helps identify performance bottlenecks and optimize the agent's response time.

In [ ]:
if 'results' in locals():
    display_metrics(results)
else:
    print("No results object found. Please run sections 5 and 6 first.")

### Interpreting the Latency Breakdown

The **latency_summary** metric is a vital tool for performance monitoring of the Flotorch Support Agent:

* **Total Latency (Root Steps)**: The cumulative execution time for all root-level operations in milliseconds. This represents the end-to-end time from query initiation to final response.
* **Average Root Step Latency**: The mean execution time across all root steps, providing a baseline for expected performance.
* **Latency Breakdown**: Detailed timing information for individual operations:
    * **invocation**: Total time for the agent invocation cycle
    * **invoke_agent**: Time spent in the agent execution framework
    * **call_llm**: Time taken for LLM API calls (can include multiple calls in a single turn)
    * **execute_tool**: Time spent executing custom tools (e.g., RAG knowledge base search)
    * Other operation-specific latencies as captured in the trace

For a Flotorch Support Agent, understanding latency breakdown helps identify:
- **LLM call bottlenecks**: If `call_llm` latencies are high, consider model optimization or caching strategies
- **RAG search performance**: If `execute_tool` (knowledge base search) is slow, optimize vector store queries or indexing
- **Overall response time**: Monitor total latency to ensure the agent delivers fast, context-aware responses using Flotorch documentation


## 13. Summary of Agent Latency Evaluation Notebook

This notebook demonstrates the professional methodology for evaluating the latency performance of a **Flotorch ADK Agent** (configured as a **Flotorch-Assistant use case** that provides context-aware responses using Flotorch documentation via RAG) using the **Flotorch Eval framework**.

**Use Case**: Flotorch-Assistant  - Provides context-aware responses using Flotorch documentation via RAG.

**Evaluation Metric**: latency_summary

## Core Process

### 1. Setup and Instrumentation
- Configure a `FlotorchADKAgent` with custom RAG tools (e.g., a knowledge base search function for Flotorch documentation).
- Enable **OpenTelemetry Tracing** via the `tracer_config`.
- This instrumentation allows detailed capture of timing and internal operations.

### 2. Execution and Data Generation
- Run a sample query through the agent using the **Runner**.
- This automatically generates an **Agent Trajectory** in the form of OpenTelemetry traces.
- The trace records the timing of each component, including:
  - LLM calls
  - RAG tool executions (knowledge base search)
  - Step-by-step agent operations

### 3. Evaluation
- Use the `AgentEvaluator` client along with the specialized **LatencyMetric** from `flotorch-eval`.
- The evaluator processes the trace data to compute latency statistics using the **latency_summary** metric.

### 4. Analysis
- The notebook displays a thorough latency breakdown, including:
  - **Total Latency (Root Steps)**
  - **Average Root Step Latency**
  - Timing for individual steps such as:
    - `invocation`
    - `call_llm`
    - `execute_tool` (RAG search operations)

## Purpose and Benefits

This evaluation provides **actionable performance metrics** that help developers:

- Identify latency bottlenecks in the Flotorch Support Agent  
- Optimize slow operations, particularly RAG search performance  
- Track performance trends over time  
- Ensure the Flotorch Support Agent delivers **fast and reliable context-aware responses** using Flotorch documentation